<h1>IS71068A Data Programming (2022)</h1>
<h2>Coursework 2</h2>

<h3>Mr Elliot Walker</h3>
<h3>PT MSc Data Science</h3>
<h3>ID: 3368 6408</h3>

<h2>Overview</h2>
<p>What is the project about?</p>

<h2>Problem Area</h2>
<p>Which topic does this report pertain to?</p>

<h2>Methods</h2>
<p>What methods are utilized in collection, preprocessing, and analysis of data. Also discuss modelling technique used for creating predictions.</p>

<h2>Definitions</h2>
<p>How is anti-Russian sentiment defined?</p>

<h2>Collection of Data</h2>
<p>Investigate occurances of anti-Russian sentiment in newspapers based on its definition.</p>

<h2>Preprocessing</h2>
<p>Conduct encoding and tidying on accumulated data.</p>

<h2>Analysis</h2>
<p>Conduct EDA on accumulated data.</p>

<h2>Modelling</h2>
<p>Create a classifier which calculates the probability of a sentence exhbiting a certain discrete value of sentiment (e.g., pro, neutral, anti).</p>

<h2>Predictions</h2>
<p>Use the model to generate predictions for sentiment across different newspapers, based on keywords in test paragraphs.</p>

<h2>Discussion</h2>

<h2>Summary</h2>

<h2>References</h2>